# Find Guider mode ROIs
Craig Lage - 11-Dec-25

In [ ]:
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.time import Time, TimeDelta
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData, getMostRecentRowWithDataBefore
from lsst.summit.utils.butlerUtils import makeDefaultButler, getExpRecordFromDataId

In [ ]:
instrument='LSSTCam'
butler = makeDefaultButler(instrument)
client = makeEfdClient()

In [ ]:
def extract_floats(text):
    # Regex pattern to match floats and integers, including signs and scientific notation
    # [-+]? : optional sign
    # \d*   : zero or more digits
    # \.?   : optional decimal point
    # \d+   : one or more digits after optional decimal
    pattern = r"[-+]?\d*\.?\d+"
    
    # Find all occurrences matching the pattern and convert them to actual float objects
    found_strings = re.findall(pattern, text)
    return [float(match) for match in found_strings]

def LogMessageTest(df):
    return df['functionName'] == 'get_guider_roi'


In [ ]:
for expId in range(2025121000063, 2025121000083):
    print(expId)
    dataId = {'exposure':expId, 'instrument': instrument}
    expRecord = getExpRecordFromDataId(butler, dataId)
    print (f"expRecord RA = {expRecord.tracking_ra}, Dec = {expRecord.tracking_dec}, SkyAngle = {expRecord.sky_angle}")
    logMessage = getMostRecentRowWithDataBefore(
        client,
        "lsst.sal.Script.logevent_logMessage",
        timeToLookBefore=expRecord.timespan.begin, 
        where=LogMessageTest
    )
    print(logMessage['message'])
    floats_list = extract_floats(logMessage['message'])
    dRA = (expRecord.tracking_ra - floats_list[0]) * 3600.0
    dDec = (expRecord.tracking_dec - floats_list[1]) * 3600.0
    dSA = ((expRecord.sky_angle%360.0) - (floats_list[2]%360.0)) * 3600.0
    print(f"Deltas in arcseconds RA = {dRA:.1f}, Dec = {dDec:.1f}, SA = {dSA:.1f}")
    print()

In [ ]:
# For the FBS, there is no get_guider_roi log message.

for expId in range(2025121000410, 2025121000411):
    print(expId)
    dataId = {'exposure':expId, 'instrument': instrument}
    expRecord = getExpRecordFromDataId(butler, dataId)
    print (f"expRecord RA = {expRecord.tracking_ra}, Dec = {expRecord.tracking_dec}, SkyAngle = {expRecord.sky_angle}")
    print(expRecord.timespan.begin.isot)
    logMessage = getMostRecentRowWithDataBefore(
        client,
        "lsst.sal.Script.logevent_logMessage",
        timeToLookBefore=expRecord.timespan.begin, 
        where=LogMessageTest
    )
    print(logMessage['message'])
    print(Time(logMessage['private_kafkaStamp'], format='unix_tai').isot)
    floats_list = extract_floats(logMessage['message'])
    dRA = (expRecord.tracking_ra - floats_list[0]) * 3600.0
    dDec = (expRecord.tracking_dec - floats_list[1]) * 3600.0
    dSA = ((expRecord.sky_angle%360.0) - (floats_list[2]%360.0)) * 3600.0
    print(f"Deltas in arcseconds RA = {dRA:.1f}, Dec = {dDec:.1f}, SA = {dSA:.1f}")
    print()